In [30]:
import pandas as pd 
def load_train_data():
    data = pd.read_csv('Acoustic_train.csv')
    return data

def load_test_data():
    data = pd.read_csv('Acoustic_test_no_class.csv')
    return data

train_data = load_train_data()
test_data = load_test_data()

In [31]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
import pandas as pd


In [32]:

# Load and prepare the data
X_train = train_data.drop(columns=["Index", "Class"])
y_train = train_data["Class"]


In [33]:

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


In [34]:

# Determine the number of clusters (equal to the number of unique classes)
num_clusters = y_train.nunique()


In [35]:

# K-Means Clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_train_scaled)


KMeans(n_clusters=4, random_state=42)

In [36]:

# Predict cluster labels
cluster_labels = kmeans.labels_

# Evaluate Clustering Performance
ari_score = adjusted_rand_score(y_train, cluster_labels)  # ARI: How well clusters align with true labels
nmi_score = normalized_mutual_info_score(y_train, cluster_labels)  # NMI: Normalized mutual information

print("Adjusted Rand Index (ARI):", ari_score)
print("Normalized Mutual Information (NMI):", nmi_score)

# Add cluster labels to the dataset for inspection
train_data["Cluster_Labels"] = cluster_labels
train_data.head()


Adjusted Rand Index (ARI): 0.3047186631441292
Normalized Mutual Information (NMI): 0.3232219413465627


,Index,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,...,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy,Cluster_Labels
0,1,sad,0.146,0.522,5.612,76.090,2.258,-0.947,0.697,-0.430,...,1.000,0.153,0.031,0.388,0.246,-0.030,2.243,0.751,0.969,0
1,2,happy,0.077,0.552,7.346,189.398,2.332,0.214,0.324,0.190,...,0.996,0.902,0.439,0.333,0.153,-0.002,1.346,0.818,0.971,2
2,3,relax,0.140,0.522,7.122,157.623,1.902,0.682,0.818,0.314,...,0.742,0.010,0.631,0.364,0.229,-0.162,2.691,0.758,0.967,0
3,4,relax,0.138,0.578,6.464,59.927,2.827,0.391,0.187,0.127,...,0.862,0.000,0.647,0.269,0.263,-0.039,0.498,0.748,0.964,3
4,5,happy,0.125,0.519,7.091,114.608,1.321,-0.336,0.529,-0.233,...,0.448,0.031,0.997,0.405,0.191,-0.019,1.922,0.818,0.969,2


In [37]:
from sklearn.metrics import accuracy_score
import numpy as np

# Map cluster labels to true labels
label_mapping = {}
for cluster in np.unique(cluster_labels):
    mask = (cluster_labels == cluster)
    true_labels = y_train[mask]
    most_common_label = true_labels.mode()[0]
    label_mapping[cluster] = most_common_label

# Map the cluster labels to the most common true labels
mapped_labels = np.vectorize(label_mapping.get)(cluster_labels)

# Calculate accuracy
accuracy = accuracy_score(y_train, mapped_labels)
print("Accuracy:", accuracy)

Accuracy: 0.5966666666666667
